# Thực nghiệm thuật toán Decision Tree trên tập dataset 2

<h2> Table of content </h2>
<div class ="alert alert-block alert-info" style="margin-top: 20px">
  <ol>
    <li><a href = "#1"> Tạo tập dữ liệu </a></li>
    <li><a href = "#2"> Thử nghiệm mô hình mặc định </a></li>
    <li><a href = "#tuning"> Tinh chỉnh siêu tham số  </a></li>
  </ol>
</div>

<h3 id = "1"> 1. Tạo tập dữ liệu </h3>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [4]:
data_train = pd.read_csv("Data2_train.csv")
data_test = pd.read_csv("Data2_test.csv")

data_train = data_train.drop(data_train.columns[0], axis = 1)
data_test = data_test.drop(data_test.columns[0], axis = 1)

data_train.head()

,GpsProvider,Market/Regular,vehicle_no,Origin_Location,Destination_Location,Planned_ETA,ontime,OriginLocation_Code,DestinationLocation_Code,trip_start_date,TRANSPORTATION_DISTANCE_IN_KM,vehicleType,customerID,supplierID,Material Shipped,Org_lat,Org_lon,Des_lat,Des_lon
0,3,0,136,23,383,173,0.0,127,169,157,80.00,34,15,92,764,16.560192,80.792293,23.704770,72.539096
1,3,0,1467,93,135,290,0.0,111,224,263,43.00,34,15,95,692,28.722966,77.345364,28.476329,77.733763
2,6,0,1188,82,92,221,0.0,127,200,200,53.90,34,15,28,229,16.560192,80.792293,12.730174,77.956202
3,3,0,559,82,70,103,0.0,153,159,93,31.00,34,15,28,271,12.992700,77.803841,12.777875,77.642276
4,3,0,909,37,161,172,0.0,76,335,154,360.15,36,15,26,578,18.970774,72.808712,21.606548,73.009534


Dữ liệu cần dự đoán ở đây là cột ontime

In [5]:
X_train = data_train.iloc[:, data_train.columns != "ontime"].values
y_train = data_train["ontime"].values

X_test = data_test.iloc[:, data_test.columns != "ontime"].values
y_test = data_test["ontime"].values

<h3 id = "2"> 2. Thử nghiệm mô hình mặc định </h3>

In [6]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       821
         1.0       0.69      0.76      0.73       161

    accuracy                           0.91       982
   macro avg       0.82      0.85      0.83       982
weighted avg       0.91      0.91      0.91       982



<h3 id = "tuning"> 3. Tinh chỉnh siêu tham số </h3>

Em tinh chỉnh các siêu tham số sau

<li> <b>max_depth </b>: Đây là số lượng tối đa của các nút trong cây quyết định. Tăng giá trị của max_depth có thể làm tăng khả năng mô hình học được các mối quan hệ phức tạp hơn trong dữ liệu, nhưng cũng có thể dẫn đến overfitting.

<li> <b>min_samples_split </b>:  Đây là số lượng mẫu tối thiểu yêu cầu để một nút có thể tiếp tục được chia thành các nút con. Giảm giá trị min_samples_split có thể làm tăng độ phức tạp của cây và có thể dẫn đến overfitting. 

<li> <b>min_samples_leaf </b>: Đây là số lượng mẫu tối thiểu yêu cầu trong mỗi lá của cây quyết định. Tương tự như min_samples_split, giảm giá trị min_samples_leaf có thể làm tăng độ phức tạp của cây, trong khi tăng giá trị có thể dẫn đến underfitting.

<li> <b> criterion </b>: Decision Tree Classifier có thể sử dụng các tiêu chí khác nhau để đo lường chất lượng chia nhánh, ví dụ như "gini" hoặc "entropy". Thông thường, không có một tiêu chí nào tốt hơn mà phù hợp cho tất cả các bài toán, nên em thể thử nghiệm cả hai để xem cái nào hoạt động tốt nhất.

Siêu tham số được tinh chỉnh bằng phương pháp đánh giá chéo (cross validaition) với việc tập dữ liệu được chia làm 10 phần không chồng chéo và có kích thước bằng nhau. 
Ngoài ra thước đo cho mô hình em chọn là F1 - score

In [7]:
from sklearn.model_selection import GridSearchCV

hyperparameters = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ["gini", "entropy"]
}

grid_search = GridSearchCV(model, hyperparameters, cv=10, scoring= 'f1_micro')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 5}


In [8]:
model = DecisionTreeClassifier(criterion= "entropy", max_depth = 9, min_samples_leaf = 10, min_samples_split = 15)
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95       821
         1.0       0.77      0.66      0.71       161

    accuracy                           0.91       982
   macro avg       0.85      0.81      0.83       982
weighted avg       0.91      0.91      0.91       982



In [9]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(25,20))
_ = plot_tree(model, filled=True)
fig.savefig("decistion_tree_2.png")